In [2]:
# Load the aiida profile
from aiida import load_profile
aiida_profile = load_profile()
aiida_profile.name

'default'

In [3]:
## Loading the some require packages and module
from aiida_kkr.workflows import (combine_imps_wc, kkr_flex_wc,
                                 kkr_imp_wc, kkr_imp_sub_wc
                                )
from aiida_kkr.calculations import KkrCalculation, KkrimpCalculation
from aiida.orm import (Group, load_group, load_node, Dict, Code
                      )
from aiida_kkr.tools import kkrparams
from aiida.engine import submit

In [4]:
# One very frequently used option
metadata_option_1 = {'max_wallclock_seconds': 36000,'resources': 
               {'tot_num_mpiprocs': 48, 'num_machines': 1},
              'custom_scheduler_commands': 
              '#SBATCH --account=jara0191\n\nulimit -s unlimited; export OMP_STACKSIZE=2g',
              'withmpi': True
                    }

In [5]:
# Note that here for doped 3d and 4d atoms into the Bi2Te3 host saved in group-74

group_74 = load_group(74)
## first node of group-74 is for kkr_imp_wc process
# is being used for the host_gf or kkr_flex files

imp_wc_with_gf = load_node(16100)
imp1_wc = group_74.nodes[1]
imp2_wc = group_74.nodes[2]

# gf_host_remote = imp_wc_with_gf.inputs.remote_data_gf
# host_gf_wc = imp_wc_with_gf.get_outgoing(node_class=kkr_flex_wc).all()[0].node


impurity1_output_node = imp1_wc.outputs.workflow_info
impurity2_output_node = imp2_wc.outputs.workflow_info
offset_imp2 = {'index':1}

In [8]:
scf_node = imp1_wc.get_outgoing(node_class=kkr_imp_sub_wc).all()[0].node
## kkr_flex_wc
host_gf = imp1_wc.inputs.remote_data_gf.get_incoming(node_class=kkr_flex_wc).all()[0].node
kkr_calc = host_gf.get_outgoing(node_class=KkrCalculation).all()[0].node
kkr_flex_kkr_param = kkr_calc.inputs.parameters

In [9]:
kkr_flex_kkr_param = kkr_calc.inputs.parameters.get_dict()
kkr_flex_kkr_param['RCLUSTZ'] *= 1
kkr_flex_kkr_param['NSHELD'] = 2500

In [11]:
settings = combine_imps_wc.get_wf_defaults()
builder = combine_imps_wc.get_builder()
settings['jij_run']= True
settings['retrieve_kkrflex'] = False
settings['mag_init']= False
settings['threshold_switch_high_accuracy'] = 1e-2
settings['convergence_criterion'] = 1e-7 #convergence_criterion
settings['threshold_aggressive_mixing']= 0.05
settings['strmix']= 0.01
settings['aggrmix'] = 0.05
settings['broyden-number'] = 20
settings['nsteps'] = 100
settings['kkr_runmax'] = 10

Version of workflow: 0.3.1


In [12]:
options = scf_node.inputs.options.get_dict()
options['max_wallclock_seconds'] = 36000*1
# options

In [10]:
builder.impurity1_output_node = impurity1_output_node
builder.impurity2_output_node = impurity2_output_node
builder.offset_imp2 = Dict(dict=offset_imp2)

# scf namespace setup
builder.scf.kkrimp = scf_node.inputs.kkrimp
builder.scf.options = Dict(dict=options)
builder.scf.wf_parameters = scf_node.inputs.wf_parameters

# host_gf namespace setup
builder.host_gf.kkr = host_gf.inputs.kkr
builder.host_gf.options = host_gf.inputs.options
builder.host_gf.params_kkr_overwrite = Dict(dict=kkr_flex_kkr_param) #host_gf.inputs.wf_parameters

builder.wf_parameters_overwrite = Dict(dict=settings)
builder.metadata.label = str(imp1_wc.label+ ':' +imp2_wc.label)

In [13]:
# To submit the builder
# combined_imp_submission = submit(builder)

In [101]:
# Note that here for doped 3d atoms into the Bi2Te3 host

group_74 = load_group(74)
group_91 = load_group(91)

## first node of group-64 is for kkr_imp_wc process
# is being used for the host_gf or kkr_flex files

imp1_wc = group_74.nodes[0]
imp2_wc = group_91.nodes[0]

scf = imp1_wc

params_kkr_overwrite = Dict(dict={'NSHELD' : 2500})
host_kkr = Code().get_from_string('kkr@claix18_init')

scf_kkrimp = Code().get_from_string('kkrflex@claix18_init')
options = Dict(dict=metadata_option_1)

imp1_output_node = imp1_wc.outputs.workflow_info
imp2_output_node = imp2_wc.outputs.workflow_info
offset_imp2 = {'index':1}



settings = combine_imps_wc.get_wf_defaults()
settings['jij_run'] = True
settings['dos_run'] = False
settings['lmdos'] = False
settings['strmix'] = 0.02
settings['aggrmix'] = 0.05
settings['threshold_aggressive_mixing'] = 0.05
settings['convergence_criterion'] = 1e-7
settings['threshold_switch_high_accuracy'] = 1e-2
settings['retrieve_kkrflex'] = False
settings['mag_init'] = False
settings['mixreduce'] = 0.5
settings['nsteps'] = 100
settings['kkr_runmax'] = 10
settings['broyden-number'] = 20



# if this function works then delete the below section
ilayer1 = imp1_wc.inputs.impurity_info.get_dict()['ilayer_center']
ilayer2 = imp2_wc.inputs.impurity_info.get_dict()['ilayer_center']

label = imp1_wc.label.split(':')[0] + ':'+imp2_wc.label + '_il_' +str(ilayer1) + '_il_'+str(ilayer2)
print('label : ',label)
submit_combine_imp(imp1_wc_node=imp1_wc, imp2_wc_node=imp2_wc, kkr_code=host_kkr, kkr_imp_code=scf_kkrimp, offset_imp2=offset_imp2, 
                   options = options, scf_wf_parameter=None, params_kkr_overwrite=params_kkr_overwrite,settings=settings, 
                   gf_host_remote=None, dry_run= False, label=None
                  )


Version of workflow: 0.3.1
label :  Sc:Sc:Bi2Te3_il_3_il_4


<WorkChainNode: uuid: 782003a4-00c7-47d1-937b-ba02093e1b13 (pk: 24510) (aiida_kkr.workflows._combine_imps.combine_imps_wc)>

In [26]:
## Here trying to work with host Au and other impurity
## THe calc is taken from the wasmer
imp_Au_group = load_group(85)
wc_list = list(imp_Au_group.nodes)


imp_wc_1 = wc_list[0]
# print('first imp wc : ', imp_wc_1.label)
imp_wc_2 = wc_list[1]
# print('econd imp wc : ', imp_wc_2.label)
# setting the scf and host namespace

scf = imp_wc_1
host_wf_parameters = Dict(dict={'NSHELD' : 2500,
                                'BZDIVIDE': [40, 40, 40],
                                'FCM': 20,
                                'EMAX': 1.2,
                                'EMIN': -0.8,
                                'GMAX': 65.0,
                                'LMAX': 3,
                                'NCHEB': 12,
                              
                               })

host_kkr = Code().get_from_string('kkr@claix18_init')
scf_kkrimp = Code().get_from_string('kkrflex@claix18_init')
options = Dict(dict=metadata_option_1)

imp1_output_node = imp_wc_1.outputs.workflow_info
imp2_output_node = imp_wc_2.outputs.workflow_info
offset_imp2 = {'index':1}

settings = combine_imps_wc.get_wf_defaults()
settings['jij_run'] = False
settings['dos_run'] = False
settings['lmdos'] = False
settings['strmix'] = 0.001
settings['aggrmix'] = 0.008
settings['threshold_aggressive_mixing'] = 0.008
settings['convergence_criterion'] = 1e-7
settings['threshold_switch_high_accuracy'] = 1e-2
settings['retrieve_kkrflex'] = False
settings['mag_init'] = False
settings['mixreduce'] = 0.5
settings['nsteps'] = 100
settings['kkr_runmax'] = 10

combine_imp_builder = combine_imps_wc.get_builder()
combine_imp_builder.impurity1_output_node = imp1_output_node
combine_imp_builder.impurity2_output_node = imp2_output_node
combine_imp_buildeFalseset_imp2 = Dict(dict=offset_imp2)

# scf namespace setup
combine_imp_builder.scf.kkrimp = scf_kkrimp
combine_imp_builder.scf.options = options
combine_imp_builder.scf.wf_parameters = scf.inputs.wf_parameters
# combine_imp_builder.gf_host_remote
# host_gf namespace setup
combine_imp_builder.host_gf.kkr = host_kkr
combine_imp_builder.host_gf.options = options
combine_imp_builder.host_gf.params_kkr_overwrite = host_wf_parameters #host_gf.inputs.wf_parameters

combine_imp_builder.wf_parameters_overwrite = Dict(dict=settings)
combine_imp_builder.metadata.label = str(imp1_wc.label+ ':' +imp2_wc.label)
# submission = submit(combine_imp_builder)

Version of workflow: 0.3.1


In [96]:
# An attemp to run a bunch of wc of the double impurity from two groups of single impurity wc or 
# two list of the single impurity wc

def submit_double_imp_wc_bunch( si_imp_list1, si_imp_list2, kkr_code, kkr_imp_code,builder_options,  
                                succ_group_label, succ_group_descr, offset_imp2={'index':1}, 
                                max_submission=20, setting=None, gf_host_remote=None,
                                scf_wf_parameter=None, params_kkr_overwrite=None, dry_run= True,
                                max_fail_wc = 10, debug= True
                              ):
    import time as t
    """
    params: 
        : Two groups : Two single imp wc groups.
        : Two lists : Two single imp wc list.
        : Settings Dict : Parameter settings.
        : Builder_options Dict : builder_option
        : succ_group_label str : Create a group by this name if it does not exist in the DB.
        : succ_group_descr str : Set the created group description either it exists or not, it not a new group
                            will be created.
    """
    
    # Create group or load existing group
    succ_group = group_not_exist_create(group_label=succ_group_label, group_descr=succ_group_descr)
    fail_group = group_not_exist_create(group_label=succ_group_label+'_fail', group_descr=succ_group_descr+'_fail')
    
    
#     test_data and prepare for further
    if not isinstance(si_imp_list1, list):
        print('The given impurity wc list 1 is not the list type.')
        return print('Please provide single_imp1_list.')
    if not isinstance(si_imp_list2, list):
        print('The given impurity wc list 2 is not the list type.')
        return print('Please provide single_imp2_list.')
    
    all_combination_dict = {}
    all_success_dict = {}
    all_submission_dict = {}
    all_resedue_dict = {}
    
    tot_wc_num = 0
    all_submission_num = 0
    all_success_num = 0
    all_failed_num = 0
    
    # Here to create all the possible dict
    for i in si_imp_list1[:]:
        for j in si_imp_list2[:]:
            node_truple = (i, j)
            pk_truple = (i.pk, j.pk)
            imp1_info = i.inputs.impurity_info.get_dict()
            imp2_info = j.inputs.impurity_info.get_dict()
            ilayer1 = str(imp1_info['ilayer_center'])
            ilayer2 = str(imp2_info['ilayer_center'])
            label= i.label.split(':')[0] + ':'+ j.label+'_il_'+ilayer1+'_il_'+ilayer2        
            all_combination_dict[tot_wc_num] = {'node_truple' : node_truple,
                                                'pk_truple' : pk_truple,
                                                'label' : label}
            tot_wc_num +=1
    
    all_resedue_dict = all_combination_dict.copy()
    
    # To start the submission process, save the success nodes and failed nodes into the corresponding
    # group
    for key, val in all_combination_dict.items():

        imp1_wc_node = all_combination_dict[key]['node_truple'][0]
        imp2_wc_node = all_combination_dict[key]['node_truple'][1]
        label = all_combination_dict[key]['label']
       
        wc_pre_exist = check_wc_exist_in_group(succ_group, wc_label=label)
        if wc_pre_exist:
            continue
        
        if all_failed_num == max_fail_wc:
            break
        
        if debug: 
            print('imp1_pk for submission', imp1_wc_node.pk)
            print('imp2_pk for submission', imp2_wc_node.pk)


        if not dry_run: 
            submission = submit_combine_imp(imp1_wc_node, imp2_wc_node, kkr_code, kkr_imp_code, offset_imp2,
                                            options, settings, dry_run, label, gf_host_remote,
                                            scf_wf_parameter, params_kkr_overwrite )
            all_submission_dict[key] = all_combination_dict[key].copy()    
            all_submission_num += 1
        while((all_submission_num - all_success_num + all_failed_num >= max_submission) or 
                (tot_wc_num - all_submission_num == 0)):
                t.sleep(60*30)
                if submission.is_finished == True:
                    if submission.exit_status == 0 : 
                        all_success_num +=1
                        print('all_success_num : ', all_success_num) 
                        all_success_dict[key] = all_combination_dict[key].copy()
                        all_resedue_dict.pop(key)
                        succ_group.add_nodes(list(submission.pk))
                    else:
                        all_failed_num += 1
                        print('all_failed_num : ', all_failed_num)
                        fail_group.add_nodes(list(submission.pk))
                        
                        if all_failed_num == max_fail_wc:
                            break
    return all_success_dict, all_resedue_dict
            

def submit_combine_imp(imp1_wc_node, imp2_wc_node, kkr_code, kkr_imp_code, offset_imp2, options, 
                       settings, dry_run, label, gf_host_remote=None,
                       scf_wf_parameter = None, params_kkr_overwrite=None):
    
    from aiida_kkr.workflows import combine_imps_wc, kkr_imp_sub_wc, kkr_flex_wc
    from aiida.orm import Dict
    imp1_output = imp1_wc_node.outputs.workflow_info
    imp2_output = imp2_wc_node.outputs.workflow_info
    if scf_wf_parameter==None:
        sub_wc1_node = imp1_wc_node.get_outgoing(node_class=kkr_imp_sub_wc).first().node
        scf_wf_parameters = sub_wc1_node.inputs.wf_parameters
    if params_kkr_overwrite==None:
        sub_gf_write_out = imp1_wc_node.get_outgoing(node_class=kkr_flex_wc).first().node
        params_kkr_overwrite = sub_gf_write_out.inputs.params_kkr_overwrite
    if settings==None:
        settings = imp1_wc_node.inputs.wf_parameters_overwrite
    if label==None:
        label = 'pk' + str(imp1_wc_node.pk)+':'+ str(imp1_wc_node.pk)
    
    builder = combine_imps_wc.get_builder()
    builder.impurity1_output_node = imp1_output
    builder.impurity2_output_node = imp2_output
    if isinstance(offset_imp2, dict):
        builder.offset_imp2 = Dict(dict=offset_imp2)
    else:
        builder.offset_imp2 = offset_imp2
    
    builder.scf.kkrimp = kkr_imp_code
    builder.scf.options = options
    builder.scf.wf_parameters = scf_wf_parameters
    
    if gf_host_remote==None:
        builder.host_gf.kkr = kkr_code
        builder.host_gf.options = options
        builder.host_gf.params_kkr_overwrite = params_kkr_overwrite #host_gf.inputs.wf_parameters
    else:
        builder.gf_host_remote = gf_host_remote
    if settings!=None:
        if isinstance(settings, dict):
            builder.wf_parameters_overwrite = Dict(dict=settings)
        else:
            builder.wf_parameters_overwrite = settings
            
    builder.metadata.label = label
    if not dry_run:
        submission = submit(builder)
        return submission
    else:
        msg = ' This is dry_run. '
        return msg
def group_not_exist_create(group_label, group_descr=None):
    from aiida.orm import load_group, Group
    """
        Check the group exist either must create
    """
    if group_descr==None:
        group_descr='No Description is added'
    
    try:
         group = load_group(group_label)
    except:
        print('Group named {} is not exist but is being created .'.format(group_label))
        group = Group(label=group_label, description=group_descr)
        group.store()
        print('Newly created group pk {}'.format(group.pk))
    return group


def check_wc_exist_in_group(group, wc_label=None, wc_pk=None):
    nodes_list = list(group.nodes)
    nodes_label = [i.label for i in nodes_list[:]]
    
    if wc_label in nodes_label:
        print('node_label-{} is exist in the group-{}.'.format(wc_label, group.label))
        return True
              
    elif wc_pk in nodes_label:
        print('node_pk-{} is exist in the group-{}.'.format(wc_label, group.label))
        return True
              
    else:
        return False
    

In [103]:
# Prepartion to submit il-3 Vs il-3 for the 3d_&_4d block

# extract from parent wc
## node 24418 is succefull wc for li-3 il-3 from node 16215 and 16237
## node 24493 is succefull wc for li-4 il-4 from node 18853 and 18853
## node 24510 is succefull wc for li-4 il-4 from node 18853 and 16215


parent_wc = load_node(24418)

kkr_imp_code = parent_wc.inputs.scf__kkrimp
options = parent_wc.inputs.scf__options
scf_wf_parameters = parent_wc.inputs.scf__wf_parameters

settings = parent_wc.inputs.wf_parameters_overwrite

gf_host_remote = parent_wc.outputs.remote_data_gf

offset_imp2 = parent_wc.inputs.offset_imp2
group_74 = load_group(74)
si_imp_list1 = list(group_74.nodes)[0:2]
si_imp_list2 = list(group_74.nodes)[0:2]

succ_group_label = '3d_4d_dope_Bi2Te3_il_3:il_3_offset_1'
succ_group_descr = 'only from group ID 74'
submit_double_imp_wc_bunch( si_imp_list1= si_imp_list1, si_imp_list2=si_imp_list2, kkr_code=None, 
                            kkr_imp_code=kkr_imp_code,builder_options=options,  
                            succ_group_label=succ_group_label, succ_group_descr=succ_group_descr,
                            offset_imp2=offset_imp2, max_submission=20, setting=settings, 
                            gf_host_remote=gf_host_remote, scf_wf_parameter=scf_wf_parameters, 
                            params_kkr_overwrite=None, dry_run= True, max_fail_wc = 10, debug= True
                              )

Group named 3d_4d_dope_Bi2Te3_il_3:il_3_offset_1_fail is not exist but is being created .
Newly created group pk 105
imp1_pk for submission 16195
imp2_pk for submission 16195
imp1_pk for submission 16195
imp2_pk for submission 16215
imp1_pk for submission 16215
imp2_pk for submission 16195
imp1_pk for submission 16215
imp2_pk for submission 16215


({},
 {0: {'node_truple': (<WorkChainNode: uuid: c42b89d5-4af1-4177-b030-d877cc8c7bd0 (pk: 16195) (aiida.workflows:kkr.imp)>,
    <WorkChainNode: uuid: c42b89d5-4af1-4177-b030-d877cc8c7bd0 (pk: 16195) (aiida.workflows:kkr.imp)>),
   'pk_truple': (16195, 16195),
   'label': 'Sc:Sc:Bi2Te3_il_3_il_3'},
  1: {'node_truple': (<WorkChainNode: uuid: c42b89d5-4af1-4177-b030-d877cc8c7bd0 (pk: 16195) (aiida.workflows:kkr.imp)>,
    <WorkChainNode: uuid: 38097eba-5ead-4e14-be9f-72d534638c47 (pk: 16215) (aiida.workflows:kkr.imp)>),
   'pk_truple': (16195, 16215),
   'label': 'Sc:Ti:Bi2Te3_il_3_il_3'},
  2: {'node_truple': (<WorkChainNode: uuid: 38097eba-5ead-4e14-be9f-72d534638c47 (pk: 16215) (aiida.workflows:kkr.imp)>,
    <WorkChainNode: uuid: c42b89d5-4af1-4177-b030-d877cc8c7bd0 (pk: 16195) (aiida.workflows:kkr.imp)>),
   'pk_truple': (16215, 16195),
   'label': 'Ti:Sc:Bi2Te3_il_3_il_3'},
  3: {'node_truple': (<WorkChainNode: uuid: 38097eba-5ead-4e14-be9f-72d534638c47 (pk: 16215) (aiida.workflo